In [25]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
from scipy import stats
pd.set_option('display.max_columns', 100)
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn import svm

### Functions

In [26]:
#pull in function to dummify columns
def dummify_columns(dataframe,var_list):
    '''
    dummifies a columns, merges with the dataframe, and drops the non-dummified column
    ------------
    dataframe: full dataframe
    variable: list of column names as string
    '''
    for vr in var_list:
        dummified_feature = pd.get_dummies(dataframe[vr], prefix=vr,drop_first=True)
        dataframe = pd.concat([dataframe,dummified_feature],axis=1,sort='False')
    dataframe = dataframe.drop(var_list, axis=1)
    return dataframe

def xy_split(dataframe,target):
    '''
    splits a dataframe into a target array and estimator dataframe
    '''
    y=dataframe[target]
    X=dataframe.drop(target, axis=1)
    return X,y

def standardize_columns(df,var_list):
    '''
    standardize a columns, merges with the dataframe, and drops the non-standardized column
    ------------
    dataframe: full dataframe
    variable: column name as string
    '''
    from sklearn.preprocessing import StandardScaler
    scaleit = StandardScaler()
    for vr in var_list:
        scaled_feature = pd.DataFrame(scaleit.fit_transform(df[[vr]]), index = df.index, columns=[vr+'__S'])
        df = pd.concat([df,scaled_feature],axis=1,sort=False)
    df.drop(columns = var_list, inplace = True)
    return df

def precision(y, X_pred):
    c = confusion_matrix(y, X_pred)
    return c[1][1]/(c[0][1]+c[1][1])

def add_miss_cols(df, missing_cols, df2):
    for var in missing_cols:
        df[var] = np.zeros(shape=(df.shape[0],1))
    df=df[df2.columns]
    return df

def precision(y, X_pred):
    c = confusion_matrix(y, X_pred)
    return c[1][1]/(c[0][1]+c[1][1])

### All Imputed Model - PDiab Population - All Predictors

In [27]:
#load in dataframe
cchd = pd.read_csv('Datasets/cchd_all_imputed_colfixed.csv')
cchd = cchd.drop('Unnamed: 0', axis=1)

In [28]:
#redefine variable dictionary
#RF_PDIAB removed
variables = {'nominal_categorical_ndummified':['MBSTATE_REC','MRACEHISP','MAR_P','DMAR','MEDUC','FRACEHISP',\
                                    'FEDUC','WIC','RF_GDIAB','RF_PHYPE','RF_GHYPE',\
                                    'RF_EHYPE','RF_PPTERM','RF_FEDRG','RF_ARTEC','DOB_MM',\
                                  'IP_GON','IP_SYPH','IP_CHLAM','IP_HEPB','IP_HEPC', 'PAY', 'SEX'],\
             'nominal_categorical_dummified': ['lrg_miss_imp'],\
           'continuous':['PRECARE','MAGER', 'FAGECOMB','PRIORTERM','PRIORLIVE','PRIORDEAD',\
                         'ILLB_R','ILOP_R','PREVIS','CIG_0','CIG_1','M_Ht_In','BMI',\
                         'WTGAIN','RF_CESARN','OEGest_Comb'],\
            'target':['CA_CCHD']}

In [29]:
#create train set
cchd2 = cchd.copy()
cchd2=cchd2.loc[cchd2.RF_PDIAB=='Y']
cchd2=cchd2.drop('RF_PDIAB', axis=1)
cchd2 = dummify_columns(cchd2, variables['nominal_categorical_ndummified'])
X,y = xy_split(cchd2, 'CA_CCHD')
X = standardize_columns(X, list(X.columns))

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler

In [30]:
#create validation set
cchd_test = pd.read_csv('Datasets/cchd_allimp_test.csv')
cchd_test = cchd_test.drop('Unnamed: 0', axis=1)
cchd_test2 = cchd_test.copy()
cchd_test2=cchd_test2.loc[cchd_test2.RF_PDIAB=='Y']
cchd_test2=cchd_test2.drop('RF_PDIAB', axis=1)
cchd_test2 = dummify_columns(cchd_test2, variables['nominal_categorical_ndummified'])
X_test,y_test = xy_split(cchd_test2, 'CA_CCHD')
X_test_standardized = standardize_columns(X_test,list(X_test.columns))

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler

/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return sel

/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8 were all converted to float64 by StandardScaler

In [31]:
#grid search SVM for accuracy
missing_cols = list(filter(lambda x: x not in list(X.columns), X_test_standardized.columns))
X = add_miss_cols(X,missing_cols,X_test_standardized)
svm_model = svm.SVC()
grid_para_svm = [
    {'C': [1, 10, 100, 1000],
     'kernel': ['poly'],
     'degree': [1, 2, 3]},
    {'C': [1, 10, 100, 1000],
     'gamma': [0.001, 0.0001],
     'kernel': ['rbf']}
]
grid_search_svm = GridSearchCV(svm_model, grid_para_svm, scoring='accuracy', cv=3, return_train_score=True,  n_jobs=-1)
grid_search_svm.fit(X, y)
print(grid_search_svm.best_params_)
print(grid_search_svm.best_estimator_.score(X,y))

{'C': 1, 'degree': 3, 'kernel': 'poly'}
0.8680351906158358


/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [32]:
#results on test set - the results are terrible
print(grid_search_svm.best_estimator_.score(X_test_standardized,y_test))
print(precision(y_test, grid_search_svm.best_estimator_.predict(X_test_standardized)))

0.014506494737840145
0.003547119164030294


In [36]:
confusion_matrix(y_test, grid_search_svm.best_estimator_.predict(X_test_standardized))

array([[  116, 10394],
       [    0,    37]])

In [33]:
#grid search SVM for accuracy
svm_modelp = svm.SVC()
grid_para_svmp = [
    {'C': [1, 10, 100, 1000],
     'kernel': ['poly'],
     'degree': [1, 2, 3]},
    {'C': [1, 10, 100, 1000],
     'gamma': [0.001, 0.0001],
     'kernel': ['rbf']}
]
grid_search_svmp = GridSearchCV(svm_modelp, grid_para_svmp, scoring='precision', cv=3, return_train_score=True,  n_jobs=-1)
grid_search_svmp.fit(X, y)
print(grid_search_svmp.best_params_)
print(grid_search_svmp.best_estimator_.score(X,y))

{'C': 100, 'degree': 1, 'kernel': 'poly'}
0.8885630498533724


/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [34]:
#results when optimizing for precision
print(grid_search_svmp.best_estimator_.score(X_test_standardized,y_test))
print(precision(y_test, grid_search_svmp.best_estimator_.predict(X_test_standardized)))

0.16459656774438228
0.0035083748302399275


In [35]:
confusion_matrix(y_test, grid_search_svmp.best_estimator_.predict(X_test_standardized))

array([[1705, 8805],
       [   6,   31]])

### All Imputed Model - PDiab Population - All Predictors

In [ ]:
#filter for all the Y Diab cases, hold out 10%, sample all the Y cases, and take 10x or 20x the no cases